# LLM Optimization Modelling Experiment

In [30]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [68]:
problem = '''A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a proportional critical element, 0.4
and 0.2, respectively. That is to say, each litre of C1 contains 0.4 l of the critical
element. To obtain GCA, the mixture must proportionally contain at least a 0.3
fraction of the critical element. Another requirement is that the quantity of the
critical element is seen in GCB, an 0.3 fraction at the most.
Furthermore, the minimum ratio of C1 with C2 in product GCC must be 0.3.

The profit expected for the sale of each litre of GCA, GCB and GCC is $120, $135
and $155, respectively.

Optimise the production planning of this firm. 
'''

## 2. Generate the mathematical model

In [69]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [70]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Production Planning

**Sets and Indices:**

* $P = \{GCA, GCB, GCC\}$: Set of products
* $C = \{C1, C2\}$: Set of components

**Parameters:**

* $a_i$: Available liters of component $i \in C$ 
    * $a_{C1} = 10,000$
    * $a_{C2} = 15,000$
* $d_j$: Minimum demand (in liters) for product $j \in P$
    * $d_{GCA} = 6,000$
    * $d_{GCB} = 7,000$
    * $d_{GCC} = 9,000$
* $e_i$: Critical element proportion in component $i \in C$
    * $e_{C1} = 0.4$
    * $e_{C2} = 0.2$
* $l_j$: Minimum required proportion of critical element in product $j \in P$
    * $l_{GCA} = 0.3$
    * $l_{GCB} = 0.3$ (maximum proportion)
* $r_{GCC}$: Minimum ratio of C1 to C2 in product GCC
    * $r_{GCC} = 0.3$
* $p_j$: Profit per liter of product $j \in P$
    * $p_{GCA} = 120$
    * $p_{GCB} = 135$
    * $p_{GCC} = 155$

**Decision Variables:**

* $x_{ij}$: Liters of component $i \in C$ used in product $j \in P$

**Objective Function (Maximize Profit):**

```
Maximize Z = ∑_{j∈P} p_j (∑_{i∈C} x_{ij})
```

**Constraints:**

1. **Component Availability:**
    * For each component $i \in C$:
    ```
    ∑_{j∈P} x_{ij} ≤ a_i
    ```

2. **Minimum Demand:**
    * For each product $j \in P$:
    ```
    ∑_{i∈C} x_{ij} ≥ d_j
    ```

3. **Critical Element Requirement for GCA:**
    ```
    ∑_{i∈C} e_i * x_{i,GCA} ≥ l_{GCA} * ∑_{i∈C} x_{i,GCA}
    ```

4. **Critical Element Requirement for GCB:**
    ```
    ∑_{i∈C} e_i * x_{i,GCB} ≤ l_{GCB} * ∑_{i∈C} x_{i,GCB}
    ```

5. **C1/C2 Ratio for GCC:**
    ```
    x_{C1,GCC} ≥ r_{GCC} * x_{C2,GCC}
    ```

6. **Non-negativity:**
    * For all $i \in C$ and $j \in P$:
    ```
    x_{ij} ≥ 0
    ```

This model can be solved using linear programming solvers to find the optimal production plan that maximizes the firm's profit while satisfying all constraints. 


## 3. Generate the pyomo code

In [71]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [72]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
from pyomo.environ import *

# Define sets and indices
P = {'GCA', 'GCB', 'GCC'}  # Products
C = {'C1', 'C2'}  # Components

# Define parameters (using your sample data)
a = {'C1': 10000, 'C2': 15000}  # Available liters of each component
d = {'GCA': 6000, 'GCB': 7000, 'GCC': 9000}  # Minimum demand for each product
e = {'C1': 0.4, 'C2': 0.2}  # Critical element proportion in each component
l = {'GCA': 0.3, 'GCB': 0.3}  # Minimum required proportion of critical element
r_GCC = 0.3  # Minimum ratio of C1 to C2 in GCC
p = {'GCA': 120, 'GCB': 135, 'GCC': 155}  # Profit per liter of each product

# Create the model
model = ConcreteModel()

# Define decision variables
model.x = Var(C, P, domain=NonNegativeReals)

# Define objective function (maximize profit)
model.profit = Objective(sum(p[j] * sum(model.x[i, j] for i in C) for j in P), sense=maximize)

# Define constraints

# 1. Component availability
def component_availability_rule(model, i):
    return sum(model.x[i, j] for j in P) <= a[i]

model.component_availability = Constraint(C, rule=component_availability_rule)

# 2. Minimum demand
def minimum_demand_rule(model, j):
    return sum(model.x[i, j] for i in C) >= d[j]

model.minimum_demand = Constraint(P, rule=minimum_demand_rule)

# 3. Critical element requirement for GCA
model.critical_element_GCA = Constraint(
    sum(e[i] * model.x[i, 'GCA'] for i in C) >= l['GCA'] * sum(model.x[i, 'GCA'] for i in C)
)

# 4. Critical element requirement for GCB
model.critical_element_GCB = Constraint(
    sum(e[i] * model.x[i, 'GCB'] for i in C) <= l['GCB'] * sum(model.x[i, 'GCB'] for i in C)
)

# 5. C1/C2 ratio for GCC
model.ratio_GCC = Constraint(model.x['C1', 'GCC'] >= r_GCC * model.x['C2', 'GCC'])

# Solve the model
solver = SolverFactory('glpk')  # You can use other solvers like 'cbc', 'gurobi', etc.
solver.solve(model)

# Print the results
if str(model.status) == 'optimal':
    print("Optimal Solution:")
    for j in P:
        print(f"Product {j}:")
        for i in C:
            print(f"  Component {i}: {model.x[i, j]():.2f} liters")
    print(f"Total Profit: {model.profit():.2f}")
else:
    print(f"Solver Status: {model.status}")
``` 


## 4. Input problem data and try running the generated code

In [73]:
from pyomo.environ import *

# Define sets and indices
P = {'GCA', 'GCB', 'GCC'}  # Products
C = {'C1', 'C2'}  # Components

# Define parameters (using your sample data)
a = {'C1': 10000, 'C2': 15000}  # Available liters of each component
d = {'GCA': 6000, 'GCB': 7000, 'GCC': 9000}  # Minimum demand for each product
e = {'C1': 0.4, 'C2': 0.2}  # Critical element proportion in each component
l = {'GCA': 0.3, 'GCB': 0.3}  # Minimum required proportion of critical element
r_GCC = 0.3  # Minimum ratio of C1 to C2 in GCC
p = {'GCA': 120, 'GCB': 135, 'GCC': 155}  # Profit per liter of each product

# Create the model
model = ConcreteModel()

# Define decision variables
model.x = Var(C, P, domain=NonNegativeReals)

# Define objective function (maximize profit)
model.profit = Objective(sum(p[j] * sum(model.x[i, j] for i in C) for j in P), sense=maximize)

# Define constraints

# 1. Component availability
def component_availability_rule(model, i):
    return sum(model.x[i, j] for j in P) <= a[i]

model.component_availability = Constraint(C, rule=component_availability_rule)

# 2. Minimum demand
def minimum_demand_rule(model, j):
    return sum(model.x[i, j] for i in C) >= d[j]

model.minimum_demand = Constraint(P, rule=minimum_demand_rule)

# 3. Critical element requirement for GCA
model.critical_element_GCA = Constraint(
    sum(e[i] * model.x[i, 'GCA'] for i in C) >= l['GCA'] * sum(model.x[i, 'GCA'] for i in C)
)

# 4. Critical element requirement for GCB
model.critical_element_GCB = Constraint(
    sum(e[i] * model.x[i, 'GCB'] for i in C) <= l['GCB'] * sum(model.x[i, 'GCB'] for i in C)
)

# 5. C1/C2 ratio for GCC
model.ratio_GCC = Constraint(model.x['C1', 'GCC'] >= r_GCC * model.x['C2', 'GCC'])

# Solve the model
solver = SolverFactory('glpk')  # You can use other solvers like 'cbc', 'gurobi', etc.
solver.solve(model)

# Print the results
if str(model.status) == 'optimal':
    print("Optimal Solution:")
    for j in P:
        print(f"Product {j}:")
        for i in C:
            print(f"  Component {i}: {model.x[i, j]():.2f} liters")
    print(f"Total Profit: {model.profit():.2f}")
else:
    print(f"Solver Status: {model.status}")

TypeError: Cannot create a Set from data that does not support __contains__.  Expected set-like object supporting collections.abc.Collection interface, but received 'SumExpression'.

## 5. Correct the code to verify model viability (optional)

In [88]:
from pyomo.environ import *

# Define sets and indices
P = {'GCA', 'GCB', 'GCC'}  # Products
C = {'C1', 'C2'}  # Components

# Define parameters (using your sample data)
a = {'C1': 10000, 'C2': 15000}  # Available liters of each component
d = {'GCA': 6000, 'GCB': 7000, 'GCC': 9000}  # Minimum demand for each product
e = {'C1': 0.4, 'C2': 0.2}  # Critical element proportion in each component
l = {'GCA': 0.3, 'GCB': 0.3}  # Minimum required proportion of critical element
r_GCC = 0.3  # Minimum ratio of C1 to C2 in GCC
p = {'GCA': 120, 'GCB': 135, 'GCC': 155}  # Profit per liter of each product

# Create the model
model = ConcreteModel()

# Define decision variables
model.x = Var(C, P, domain=NonNegativeReals)

total_profit_expr = sum(p[j] * sum(model.x[i, j] for i in C) for j in P)  #### HUMAN ADDED

# Define objective function (maximize profit)
model.profit = Objective(expr=total_profit_expr, sense=maximize)

# Define constraints

# 1. Component availability
def component_availability_rule(model, i):
    return sum(model.x[i, j] for j in P) <= a[i]

model.component_availability = Constraint(C, rule=component_availability_rule)

# 2. Minimum demand
def minimum_demand_rule(model, j):
    return sum(model.x[i, j] for i in C) >= d[j]

model.minimum_demand = Constraint(P, rule=minimum_demand_rule)

crit_element_gca_expr = sum(e[i] * model.x[i, 'GCA'] for i in C) >= l['GCA'] * sum(model.x[i, 'GCA'] for i in C) #### HUMAN ADDED

# 3. Critical element requirement for GCA
model.critical_element_GCA = Constraint(expr = crit_element_gca_expr)

crit_element_gcb_expr = sum(e[i] * model.x[i, 'GCB'] for i in C) <= l['GCB'] * sum(model.x[i, 'GCB'] for i in C) #### HUMAN ADDED

# 4. Critical element requirement for GCB
model.critical_element_GCB = Constraint(expr = crit_element_gcb_expr)

c1c2_ratio_expr = model.x['C1', 'GCC'] >= r_GCC * model.x['C2', 'GCC']
# 5. C1/C2 ratio for GCC
model.ratio_GCC = Constraint(expr=c1c2_ratio_expr)

# Solve the model
solver = SolverFactory('glpk')  # You can use other solvers like 'cbc', 'gurobi', etc.
solver.solve(model)

# Print the results

print("Optimal Solution:")
for j in P:
    print(f"Product {j}:")
    for i in C:
        print(f"  Component {i}: {model.x[i, j]():.2f} liters")
print(f"Total Profit: {model.profit():.2f}")



Optimal Solution:
Product GCA:
  Component C2: 2269.23 liters
  Component C1: 3730.77 liters
Product GCB:
  Component C2: 3500.00 liters
  Component C1: 3500.00 liters
Product GCC:
  Component C2: 9230.77 liters
  Component C1: 2769.23 liters
Total Profit: 3525000.00


## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [82]:
print(response.text)

## Mathematical Optimization Model for Production Planning

**Sets and Indices:**

* $P = \{GCA, GCB, GCC\}$: Set of products
* $C = \{C1, C2\}$: Set of components

**Parameters:**

* $a_i$: Available liters of component $i \in C$ 
    * $a_{C1} = 10,000$
    * $a_{C2} = 15,000$
* $d_j$: Minimum demand (in liters) for product $j \in P$
    * $d_{GCA} = 6,000$
    * $d_{GCB} = 7,000$
    * $d_{GCC} = 9,000$
* $e_i$: Critical element proportion in component $i \in C$
    * $e_{C1} = 0.4$
    * $e_{C2} = 0.2$
* $l_j$: Minimum required proportion of critical element in product $j \in P$
    * $l_{GCA} = 0.3$
    * $l_{GCB} = 0.3$ (maximum proportion)
* $r_{GCC}$: Minimum ratio of C1 to C2 in product GCC
    * $r_{GCC} = 0.3$
* $p_j$: Profit per liter of product $j \in P$
    * $p_{GCA} = 120$
    * $p_{GCB} = 135$
    * $p_{GCC} = 155$

**Decision Variables:**

* $x_{ij}$: Liters of component $i \in C$ used in product $j \in P$

**Objective Function (Maximize Profit):**

```
Maximize Z = ∑

In [83]:
print(response2.text)

```python
from pyomo.environ import *

# Define sets and indices
P = {'GCA', 'GCB', 'GCC'}  # Products
C = {'C1', 'C2'}  # Components

# Define parameters (using your sample data)
a = {'C1': 10000, 'C2': 15000}  # Available liters of each component
d = {'GCA': 6000, 'GCB': 7000, 'GCC': 9000}  # Minimum demand for each product
e = {'C1': 0.4, 'C2': 0.2}  # Critical element proportion in each component
l = {'GCA': 0.3, 'GCB': 0.3}  # Minimum required proportion of critical element
r_GCC = 0.3  # Minimum ratio of C1 to C2 in GCC
p = {'GCA': 120, 'GCB': 135, 'GCC': 155}  # Profit per liter of each product

# Create the model
model = ConcreteModel()

# Define decision variables
model.x = Var(C, P, domain=NonNegativeReals)

# Define objective function (maximize profit)
model.profit = Objective(sum(p[j] * sum(model.x[i, j] for i in C) for j in P), sense=maximize)

# Define constraints

# 1. Component availability
def component_availability_rule(model, i):
    return sum(model.x[i, j] for j in